In [4]:
# Interesting links: 

# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
# https://medium.com/swlh/sentiment-classification-for-restaurant-reviews-using-tf-idf-42f707bfe44d
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
# https://medium.com/analytics-vidhya/complete-guide-to-machine-learning-evaluation-metrics-615c2864d916

In [5]:
pip install nltk pycld2 wordsegment autocorrect

     |████████████████████████████████| 41.4MB 87kB/s 
     |████████████████████████████████| 4.8MB 34.0MB/s 
     |████████████████████████████████| 1.8MB 40.3MB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp36-cp36m-linux_x86_64.whl size=9833501 sha256=0b184c38e436333bbfb2eeeb4621404e68f701f004449ea8146c980455b7f95c
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
  Created wheel for autocorrect: filename=autocorrect-2.1.0-cp36-none-any.whl size=1811979 sha256=097a8435f7dcaa057c0c12411f5880a79039a2ec348f11a4f6a72725a008809e
  Stored in directory: /root/.cache/pip/wheels/d1/33/69/a42db65d595bb685429c788ab45e11aa2a4b6549298ee38e02
Successfully built pycld2 autocorrect


In [6]:
import nltk
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
twitter_samples.fileids()
positive_tweets = [t for t in twitter_samples.strings("positive_tweets.json")]
negative_tweets = [t for t in twitter_samples.strings("negative_tweets.json")] 

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus).toarray()
print(vectorizer.get_feature_names())
print(X.shape)
print(X)

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
(4, 9)
[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


In [8]:
print(len(positive_tweets))
print(len(negative_tweets))

5000
5000


In [9]:
train_pos_tweets = positive_tweets[:3500]
test_pos_tweets = positive_tweets[3500:]
train_neg_tweets = negative_tweets[:3500]
test_neg_tweets = negative_tweets[3500:]

In [10]:
print(train_pos_tweets[0])

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


In [11]:
print(train_neg_tweets[7])

@f0ggstar @stuartthull work neighbour on motors. Asked why and he said hates the updates on search :( http://t.co/XvmTUikWln


In [12]:
print(len(train_pos_tweets))
print(len(test_pos_tweets))
print(len(train_neg_tweets))
print(len(test_neg_tweets))

3500
1500
3500
1500


In [13]:
print(len(train_pos_tweets + train_neg_tweets))
print(len(test_pos_tweets + test_neg_tweets))

7000
3000


In [14]:
# import pycld2 as cld2

# def extract_engTweets(tweets):
#     en_tweets = []
#     for e in tweets:
#       details = cld2.detect(e)
#       # Extracts only the string with lang name i.e: 'en'
#       if details[2][0][1] == 'en': 
#         en_tweets.append(e)
#     return en_tweets

# en_train_pos_tweets = extract_engTweets(train_pos_tweets)
# en_train_neg_tweets = extract_engTweets(train_neg_tweets)
# en_test_pos_tweets = extract_engTweets(test_pos_tweets)
# en_test_neg_tweets = extract_engTweets(test_neg_tweets)
# print(len(train_pos_tweets))
# print(len(en_train_pos_tweets))
# print(len(train_neg_tweets))
# print(len(en_train_neg_tweets))
# print(len(test_pos_tweets))
# print(len(en_test_pos_tweets))
# print(len(test_neg_tweets))
# print(len(en_test_neg_tweets))


In [15]:
#pre processing
import re
import wordsegment
import autocorrect
from wordsegment import load, segment
load()
from autocorrect import Speller
spell=Speller()
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")
tknzr = TweetTokenizer()

punctuations = '''!()-![]{};:+'"\,<>./?@#$%^&*_~'''
testTweet1='@PERKSOFNIALLJH RTed! Goood Luck :)'
testTweet2='My legs hurt so bad :))))))))))))))))))'
testTweet3='#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris trouble for being top engaged members in my community this week :)'
testTweet4='@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!'

def pre_process (tweets):
  result_tweets = []
  for tweet in tweets:  
    # print(f'raw tweet: {tweet}')
    tweet = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', "", tweet, flags=re.MULTILINE) # to remove links that start with HTTP/HTTPS in the tweet
    tweet = re.sub(r'[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', "", tweet, flags=re.MULTILINE) # to remove other url links
    tweet = re.sub(r"RT", ' ', tweet, flags=re.MULTILINE) #remove 'RT'
    tweet = re.sub(r"#(\w+)", ' ', tweet, flags=re.MULTILINE) #remove '#'
    tweet = re.sub(r"@(\w+)", ' ', tweet, flags=re.MULTILINE) #remove '@'
    tweet = ''.join([i for i in tweet if not i in punctuations]) #remove ponctuations
    tweet = ' '.join(segment(tweet)) #Remove repeated letters
    tweet = ' '.join([spell(w) for w in tweet.split()]) #Avoiding misspellings and slang words
    tweet = re.sub(r"\d", "", tweet) #remove digits
    tweet = tweet.lower()
    tokens = tknzr.tokenize(tweet) #tokenize
    tokens = [word for word in tokens if not word in stopwords.words()]
    tweet = (" ").join(stemmer.stem(token) for token in tokens)
    # print(f'preprocessed tweet: {tweet}')
    result_tweets.append(tweet)
  
  return result_tweets

# proc_en_train_pos_tweets = pre_process([testTweet1,testTweet2,testTweet3,testTweet4])
# proc_en_train_pos_tweets = pre_process([en_train_pos_tweets[0]])
# proc_en_train_neg_tweets = pre_process([en_train_neg_tweets[0]])
proc_train_pos_tweets = pre_process(train_pos_tweets)
proc_train_neg_tweets = pre_process(train_neg_tweets)
proc_test_pos_tweets = pre_process(test_pos_tweets)
proc_test_neg_tweets = pre_process(test_neg_tweets)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
print(len(train_pos_tweets))
print(len(test_pos_tweets))
print(len(train_neg_tweets))
print(len(test_neg_tweets))

3500
1500
3500
1500


In [17]:
import numpy as np
proc_train_tweets = np.concatenate((proc_train_pos_tweets, proc_train_neg_tweets))
proc_test_tweets = np.concatenate((proc_test_pos_tweets, proc_test_neg_tweets))
proc_pos_tweets = np.concatenate((proc_train_pos_tweets, proc_test_pos_tweets))
proc_neg_tweets = np.concatenate((proc_train_neg_tweets, proc_test_neg_tweets))

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer(min_df=5, max_df=0.75, ngram_range=(1,3), max_features=10000)
vectorizer = TfidfVectorizer(min_df=1, max_df=0.75, ngram_range=(1,3), max_features=10000)
features_train_pos = vectorizer.fit_transform(proc_train_pos_tweets).toarray()
features_train_neg = vectorizer.fit_transform(proc_train_neg_tweets).toarray()
features_test_pos = vectorizer.fit_transform(proc_test_pos_tweets).toarray()
features_test_neg = vectorizer.fit_transform(proc_test_neg_tweets).toarray()
################################################################################
features_train = vectorizer.fit_transform(proc_train_tweets).toarray()
features_test = vectorizer.fit_transform(proc_test_tweets).toarray()
features_pos = vectorizer.fit_transform(proc_pos_tweets).toarray()
features_neg = vectorizer.fit_transform(proc_neg_tweets).toarray()
# print(f'features_train_pos: {features_train_pos}')
# print(f'type(features_train_pos): {type(features_train_pos)}')

In [19]:
print(f'features_train.shape: {features_train.shape}')
print(f'features_test.shape: {features_test.shape}')
print(f'features_train.shape: {features_pos.shape}')
print(f'features_test.shape: {features_neg.shape}')
print(f'features_train_pos.shape: {features_train_pos.shape}')
print(f'features_train_neg.shape: {features_train_neg.shape}')
print(f'features_test_pos.shape: {features_test_pos.shape}')
print(f'features_test_neg.shape: {features_test_neg.shape}')

features_train.shape: (7000, 10000)
features_test.shape: (3000, 10000)
features_train.shape: (5000, 10000)
features_test.shape: (5000, 10000)
features_train_pos.shape: (3500, 10000)
features_train_neg.shape: (3500, 10000)
features_test_pos.shape: (1500, 10000)
features_test_neg.shape: (1500, 10000)


In [20]:
# import numpy as np
# all_tweets = np.concatenate((features_train_pos,features_train_neg,features_test_pos,features_test_neg))
# train_features = np.concatenate((features_train_pos,features_train_neg))
# test_features = np.concatenate((features_test_pos,features_test_neg))

In [21]:
## Scoring and labeling the tweets
# First calculate the overall polarity score
# Assume only positive and negative
# Classify via the number of neg/pos words and emoticons

# 'tweet is expected to be an integer iterable' 
def polarity_score(tweet):
  score = 0
  for feature in tweet:
    score += feature
  return score

# 'pol_score' is expected to be a scalar
def determine_sentiment(pol_score):
  # Highly positive: +2 
  if pol_score >= 2 and pol_score <= 4:
    return 2 
  # Moderate positive: +1 
  if pol_score > 0 and pol_score < 2:
    return 1
  # Moderate negative: -1 
  if pol_score > -2 and pol_score < 0:
    return -1 
  # Highly negative: -2 
  if pol_score >= -4 and pol_score <= -2:
    return -2
  if pol_score == 0: 
    return 0
  else:#Tirar depois
    # return 2
    return 0

In [22]:
## Testing the balancing and scoring
# Classify the tweets
def score_tweets(tweets):
  tweet, length = tweets.shape
  tweets_class = []
  for i in range(tweet):
    score = polarity_score(tweets[i,:])
    sentiment = determine_sentiment(score)
    tweets_class.append(sentiment)
  return tweets_class

# all_tweets_classes = np.array(score_tweets(all_tweets))
train_pos_classes = np.array(score_tweets(features_train_pos))
train_neg_classes = np.negative(np.array(score_tweets(features_train_neg)))
test_pos_classes = np.array(score_tweets(features_test_pos))
test_neg_classes = np.negative(np.array(score_tweets(features_test_neg)))
pos_classes = np.array(score_tweets(features_pos))
neg_classes = np.negative(np.array(score_tweets(features_neg)))
train_classes = np.concatenate((train_pos_classes, train_neg_classes))
test_classes = np.concatenate((test_pos_classes, test_neg_classes))
# train_classes = np.array(score_tweets(features_train))
# test_classes = np.array(score_tweets(features_test))
all_tweets_classes = np.concatenate((train_classes, test_classes)) 
print(all_tweets_classes.size)

10000


In [23]:
# #Table 4 - Polarity of tweets
# high_positive = 0
# moderate_positive = 0
# neutral = 0
# moderate_negative = 0
# high_negative = 0

# for t in all_tweets_classes:
#   if t == 2:
#     high_positive += 1
#   if t == 1:
#     moderate_positive += 1
#   if t == 0:
#     neutral += 1
#   if t == -1:
#     moderate_negative += 1
#   if t == -2:
#     high_negative += 1

# hp_ratio = (high_positive/all_tweets_classes.size)*100
# mp_ratio = (moderate_positive/all_tweets_classes.size)*100
# neutral_ratio = (neutral/all_tweets_classes.size)*100
# mn_ratio = (moderate_negative/all_tweets_classes.size)*100
# hn_ratio = (high_negative/all_tweets_classes.size)*100

# print(f'high_positive: {high_positive} ({hp_ratio}%)')
# print(f'moderate_positive: {moderate_positive} ({mp_ratio}%)')
# print(f'neutral: {neutral} ({neutral_ratio}%)')
# print(f'moderate_negative: {moderate_negative} ({mn_ratio}%)')
# print(f'high_negative: {high_negative} ({hn_ratio}%)')

In [24]:
#Table 4 - Polarity of positive tweets
high_positive = 0
moderate_positive = 0
part_pos_neutral = 0

pos_train_classes = np.array(score_tweets(features_pos))

for t in pos_train_classes:
  if t == 2:
    high_positive += 1
  if t == 1:
    moderate_positive += 1
  if t == 0:
    part_pos_neutral += 1

print(f'high_positive: {high_positive}')
print(f'moderate_positive: {moderate_positive}')
print(f'part_pos_neutral: {part_pos_neutral}')

high_positive: 2480
moderate_positive: 2000
part_pos_neutral: 520


In [25]:
#Table 4 - Polarity of negative tweets
moderate_negative = 0
high_negative = 0
part_neg_neutral = 0

neg_train_classes = np.array(score_tweets(features_neg))

for t in neg_train_classes:
  if t == 2:
    high_negative += 1
  if t == 1:
    moderate_negative += 1
  if t == 0:
    part_neg_neutral += 1

print(f'high_negative: {high_negative}')
print(f'moderate_negative: {moderate_negative}')
print(f'part_neg_neutral: {part_neg_neutral}')

high_negative: 2435
moderate_negative: 2119
part_neg_neutral: 446


In [26]:
high_positive + high_negative + moderate_positive + moderate_negative + part_pos_neutral + part_neg_neutral

10000

In [27]:
# Gambiarra table 1
neutral = part_pos_neutral + part_neg_neutral

hp_ratio = (high_positive/all_tweets_classes.size)*100
mp_ratio = (moderate_positive/all_tweets_classes.size)*100
neutral_ratio = (neutral/all_tweets_classes.size)*100
mn_ratio = (moderate_negative/all_tweets_classes.size)*100
hn_ratio = (high_negative/all_tweets_classes.size)*100

print(f'high_positive: {high_positive} ({hp_ratio}%)')
print(f'moderate_positive: {moderate_positive} ({mp_ratio}%)')
print(f'neutral: {neutral} ({neutral_ratio}%)')
print(f'moderate_negative: {moderate_negative} ({mn_ratio}%)')
print(f'high_negative: {high_negative} ({hn_ratio}%)')

high_positive: 2480 (24.8%)
moderate_positive: 2000 (20.0%)
neutral: 966 (9.66%)
moderate_negative: 2119 (21.19%)
high_negative: 2435 (24.349999999999998%)


In [28]:
# Decision tree implementation
from sklearn import tree

decision_tree = tree.DecisionTreeClassifier()
# X = features_train, y = train_classes

# features_train = vectorizer.fit_transform(proc_train_tweets).toarray()
# train_classes = np.concatenate((train_pos_classes, train_neg_classes))
# train_pos_classes = np.array(score_tweets(features_train_pos))
# train_neg_classes = np.negative(np.array(score_tweets(features_train_neg)))

decision_tree.fit(features_train, train_classes)
y_predict = decision_tree.predict(features_test)

# decision_tree.fit(train_features, train_classes)
# y_predict = decision_tree.predict(test_features)

In [29]:
# # Random forest implementation
# from sklearn.ensemble import RandomForestClassifier

# random_forest = RandomForestClassifier(n_jobs=-1)
# random_forest.fit(features_train, train_classes)
# y_predict = random_forest.predict(features_test)

In [30]:
# # SVR implementation
# from sklearn.svm import SVR
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler

# svr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
# svr.fit(features_train, train_classes)
# y_predict = svr.predict(features_test)

In [31]:
# # Multinomial Logistic Regression (Softmax)
# from sklearn.linear_model import LogisticRegression

# softmax = LogisticRegression(multi_class='multinomial')
# softmax.fit(features_train, train_classes)
# y_predict = softmax.predict(features_test)

In [32]:
print(test_classes.shape)
print(y_predict.shape)
print(features_test.shape)

(3000,)
(3000,)
(3000, 10000)


In [33]:
from sklearn.metrics import classification_report
y_true = test_classes
y_pred = y_predict 
target_names = ['High Negative', 'Moderate Negative', 'Neutral', 'Moderate Positive', 'High Positive']
print(classification_report(y_true, y_pred, target_names=target_names))

                   precision    recall  f1-score   support

    High Negative       0.22      0.05      0.08       711
Moderate Negative       0.13      0.08      0.10       461
          Neutral       0.22      0.69      0.34       726
Moderate Positive       0.19      0.06      0.09       458
    High Positive       0.23      0.07      0.10       644

         accuracy                           0.21      3000
        macro avg       0.20      0.19      0.14      3000
     weighted avg       0.21      0.21      0.15      3000



In [34]:
# # 10-fold cross-validation
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score

# # prepare the cross-validation procedure
# cv = KFold(n_splits=10, random_state=1, shuffle=True)
# # clf = decision_tree ou random_forest ou svr ou softmax
# scores = cross_val_score(clf, features_train, train_classes, scoring='accuracy', cv=cv, n_jobs=-1)
# # Report performance
# print(f'scores.shape{scores.shape}')
# print(f'scores{scores}')